# Comparing Annual Changes in Sea Surface Temperature and Polar Sea Ice Concentrations 

## EDS 220, Fall 2021



## Authors

- **Peter Menzies**, UC Santa Barbara (petermenzies@bren.ucsb.edu) <br>
https://petermenzies.github.io/


- **Julie Cohen**, UC Santa Barbara (jscohen@bren.ucsb.edu) <br>
https://julietcohen.github.io/


- **Ryan Munnikhuis**, UC Santa Barbara (Rmunnikhuis@ucsb.edu) <br>
https://ryanmunnikhuis.github.io/


## Table of Contents

[1. Purpose](#purpose)

[2. Use Case Examples](#usecases)

[3. Datasets](#overview)

[4. Data I/O](#io)

[5. References](#references)

<a id='purpose'></a> 
### Purpose

While static maps are useful for oberserving conditons at a single point in time (or an average of a time span), it can sometimes be helpful to visualize changes in geospatial data from year to year. This notebook is intended to provide you with multiple examples of how to animate a timeseries. This lab will also show you one method to plot two seperate datasets on the same graph to compare data.  

For this exercise, we'll be subsetting and visualsing sea surface temperatures (SST) from the NOAA Optimum Interpolation 1/4 Degree Daily Sea Surface Temperature (OISST) Analysis, Version 2) and sea ice concentrations (SIC) data from the Monthly Mean Hadley Centre Sea Ice and SST dataset version 1 (HadISST1) data set. 

At the end of the lab, we will ideally have: 

- A gif showing changes of sea surface temperatures over a 40-year timeframe; 


- A gif showing of changes of sea ice concentrations over a 40-year timeframe; and


- A graph showing the trend in sea surface temperatures and sea ice concentrations over time.

<a id='usecases'></a> 
### Use Case Examples 

Gifs are an effective tool in conveying a high density of information in a short amount of time. In the context of geospatial data, animated maps are often more effective at visualizing rates of change of information compared to static maps. For example, a gif showing the movement of a hurricane towards the Eastern Seabord would likely instill a greater sense of urgency in the public to be better prepared when the storm hits landfall than a static image of a tropical cyclone. In addition, gifs often require little to no preamble to understand what is being visualized. This ease of understanding increases inclusivity and can minimize room for misinterpretation. Lastly, geospatial gifs can also be used to help predict trends in spatial data. Having a way to quickly and effectively generate an animated spatial map can help contextualize data and give researchers a sense of what future data should look like.

<a id='overview'></a> 
### Datasets

In this lab, we'll be using data from two sources. 
- SST: NOAA's 1/4° Daily Optimum Interpolation Sea Surface Temperature (OISST) version 2, or OISSTv2
- SIC: Monthly Mean Hadley Centre Sea Ice and SST dataset version 1 (HadISST1)

Data Google Drive: https://drive.google.com/drive/folders/1bdVrH6EB0JAjreb0xAbcb6Nps5oyW2eW?usp=sharing

A description of each dataset is given below: 

### Sea Surface Temperature Data 

The NOAA 1/4° daily Optimum Interpolation Sea Surface Temperature (daily OISST) Climate Data Record (CDR) provides complete ocean temperature fields constructed by combining bias-adjusted observations from different platforms (satellites, ships, buoys) on a regular global grid, with gaps filled in by interpolation. The main input source is satellite data from the Advanced Very High-Resolution Radiometer (AVHRR), which provides high temporal-spatial coverage from late 1981 to–present. 
This dataset is stored as NetCDF files (.nc files) that include latitude and longitude coordinates, temperatures, and dates.

Data source and metadata: NOAA National Centers for Environemntal Information https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00844/html

NOAA Sea Surface Temperature Optimum Interpolation methods: https://www.ncei.noaa.gov/products/climate-data-records/sea-surface-temperature-optimum-interpolation


### Sea Ice Concentration 

The Monthly Mean Hadley Centre Sea Ice and SST dataset version 1 (HadISST1) is one half of the merged Hadley-OI sea surface temperature (SST) and sea ice concentration (SIC) dataset. The merged product provides monthly global mean sea surface temperature and sea ice concentration data from 1870 to the present. The merging procedure was designed to take full advantage of the higher-resolution SST information found in version 2 of the NOAA weekly optimum interpolation (OI) SST analysis, the other half of the merged dataset. The combined dataset blends historical SST and modern SST observations from ships, buoys, drifters, and sea ice observations, partly from historical ship- and air-borne and partly from satellite data.This dataset is stored as NetCDF files (.nc files) that include latitude and longitude coordinates, temperatures, and dates.

Data source and metadata: UCAR/NCAR - DASH Repository
https://dashrepo.ucar.edu/dataset/158_asphilli.html

Journal of Climate article explaining why the datsets were merged: https://journals.ametsoc.org/view/journals/clim/21/19/2008jcli2292.1.xml

## Dataset Input/Output

### Configure Environment

We will be using xarray, rasterio, pandas, matplotlib, rioxarray, imaegio, and glob Python packages.

In [ ]:
#Configure Python Environment
import xarray as xr
import rasterio as rio
import pandas as pd 
from rasterio.plot import show
import matplotlib.pyplot as plt
import rioxarray
import imageio
import glob

### Dataset Parameters 

We have preloaded subsetted data into binder to avoid memory conflicts as well as to minimize the time needed downloading data. We'll look at a 40-year timeframe for both sea surface temperatures and sea ice concentrations. Since we've already (briefly) gone over how to visualize SST in our last presentation, we'll just show you the code used for creating this gif and go through in detail how to create an animated ice cap time series and visualize two datasets on one chart.

## Part One: Sea Surface Temperature 

In [ ]:
# Example commands to open a single NetCDF file and view the metadata

# use xarray to load in the datafile
data_example = xr.open_dataset('Data/avhrr-only-v2.19810901.nc')
# use head() to view the metadata associated with this file
data_example.head()

In [ ]:
# List of years that data were collected to read in .nc files and to title maps later on
years = range(1981, 2022)

# Empty list to populate with tif file names
images = []

In [ ]:
# Loop to read in datasets and and export as rasterized tiffs
for i in years:
    
    dataset = 'Data/oisst-avhrr-v02r01.' + str(i) + '1017.nc'
    
    nc = xr.open_dataset(dataset)
    
    nc_projected = nc.rio.write_crs(4326, inplace=True)
    
    sst = nc_projected["sst"].rio.set_spatial_dims('lon', 'lat')
    
    sst = sst.rename({'lat':'latitude', 'lon':'longitude'})
    
    sst['time'] = pd.to_datetime(sst['time'], format='%Y%m%d')
    
    sst = sst.squeeze()
    
    filename = 'Data/noaa_sst_' + str(i) + '.tif'
    
    sst.rio.to_raster(filename)
    
    images.append(filename)

In [ ]:
# Function to render the tiffs we exported
def raster_show(raster):
    
    open_rast = rio.open(raster)
    
    plt.figure(figsize=(10,10), dpi=150)
    
    plt.imshow(open_rast.read(1),
               origin='lower',
               cmap='plasma',
               extent=[-180, 180, -90, 90])

In [ ]:
raster_show('Data/noaa_sst_1981.tif')

In [ ]:
raster_show('Data/noaa_sst_2021.tif')

In [ ]:
png_rasters = []

In [ ]:
 # Loop to export as png and create list of png file names
 for i in range(0, 41):
    raster_show(images[i])
    png = images[i] + '.png'
    plt.title(years[i])
    plt.savefig(png)
    png_rasters.append(png)

In [ ]:
# Created composite gif from pngs
png_imageio = []

for filename in png_rasters:
    png_imageio.append(imageio.imread(filename))
imageio.mimsave('Data/sst_81_21.gif', png_imageio, **{'duration':0.4})

## Part Two: Sea Ice Concentrations 

In [ ]:
ds_ice = xr.open_dataset("Data/MODEL.ICE.HAD187001-198110.OI198111-202109.nc")

In [ ]:
ice = ds_ice.rio.write_crs(4326, inplace=True)

In [ ]:
ice_time = pd.DataFrame(columns=['year', 'ice_conc'])

In [ ]:
ice_year = []

ice_conc = []

In [ ]:
ice_time['year'] = ice_year
ice_time['ice_conc'] = ice_conc

In [ ]:
ice_time

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(ice_time['year'], ice_time['ice_conc'])
plt.xlabel('Year')
plt.ylabel('Ice concentration')

In [ ]:
for i in range(0, 40):
    
    ice_i = float(ice['SEAICE'][1341 + (12 * i)].mean())

    year_i = 1981 + i

    ice_year.append(year_i)

    ice_conc.append(ice_i)

## Part Three: Comparing Datasets 

<a id='references'></a> 
### References

List relevant references. Here are some additional resources on creating professional, shareable notebooks you may find useful:

1. NOAA. 2021a. *NOAA Optimum Interpolation 1/4 Degree Daily Sea Surface Temperature (OISST) Analysis, Version 2* https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00844/html
2. NOAA. 2021b. *Sea Surface Temperature - Optimum Interpolation CDR* https://www.ncei.noaa.gov/products/climate-data-records/sea-surface-temperature-optimum-interpolation
3. NCAR UCAR. 2021. *Merged Hadley-OI sea surface temperature and sea ice concentration data set* https://dashrepo.ucar.edu/dataset/158_asphilli.html
4. Journal of Climate. 2006. *A New Sea Surface Temperature and Sea Ice Boundary Dataset for the Community Atmosphere Model* https://journals.ametsoc.org/view/journals/clim/21/19/2008jcli2292.1.xml